<a href="https://colab.research.google.com/github/Karthika506/Familial_melanoma/blob/main/Familial_melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def load_dataset(data_dir):
  images = []
  masks = []
  for filename in os.listdir(data_dir):
    if filename.endswith('.jpg'):
      image_path = os.path.join(data_dir, filename)
      image = cv2.imread(image_path)
      images.append(image)

      mask_filename = filename.replace('.jpg', '_mask.jpg')
      mask_path = os.path.join(data_dir, mask_filename)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
      masks.append(mask)
  return np.array(images), np.array(masks)

def preprocess_data(images, masks):
  images = images.astype(np.float32) / 255.0
  masks = np.expand_dims(masks, axis=-1)
  return images, masks

data_dir = '/content/drive/MyDrive/familial_melanoma_dataset/DermMel/train_sep/Melanoma'
images, masks = load_dataset(data_dir)

images, masks = preprocess_data(images, masks)

In [10]:
print(images, masks)

[[[[0.5411765  0.46666667 0.57254905]
   [0.5529412  0.47843137 0.58431375]
   [0.5686275  0.49019608 0.5803922 ]
   ...
   [0.34509805 0.26666668 0.35686275]
   [0.32941177 0.25882354 0.34901962]
   [0.3254902  0.2509804  0.34901962]]

  [[0.5568628  0.4745098  0.57254905]
   [0.5529412  0.47843137 0.5764706 ]
   [0.5764706  0.49803922 0.5882353 ]
   ...
   [0.34509805 0.27058825 0.36862746]
   [0.34117648 0.27058825 0.36078432]
   [0.33333334 0.25882354 0.35686275]]

  [[0.56078434 0.48235294 0.57254905]
   [0.5647059  0.49803922 0.5803922 ]
   [0.5803922  0.49803922 0.59607846]
   ...
   [0.34901962 0.27450982 0.37254903]
   [0.3372549  0.27450982 0.36862746]
   [0.3254902  0.26666668 0.34901962]]

  ...

  [[0.11372549 0.07058824 0.10196079]
   [0.11372549 0.07058824 0.10196079]
   [0.13725491 0.08627451 0.11764706]
   ...
   [0.10196079 0.07058824 0.11372549]
   [0.09803922 0.07450981 0.11764706]
   [0.10196079 0.07058824 0.11372549]]

  [[0.10196079 0.07058824 0.09803922]
   [0.1

In [11]:
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

In [14]:
def unet_model(input_shape):
  inputs = Input(input_shape)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv1)
  pool1 = MaxPooling2D(pool_size = (2,2))(conv1)

  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv2)
  pool2 = MaxPooling2D(pool_size = (2,2))(conv2)

  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv3)
  up1 = UpSampling2D(size = (2,2))(conv3)

  concat1 = concatenate([conv2, up1], axis = -1)
  conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same')(concat1)
  conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv4)
  up2 = UpSampling2D(size = (2,2))(conv4)

  concat2 = concatenate([conv1, up2], axis = -1)
  conv5 = Conv2D(64, 3, activation = 'relu', padding = 'same')(concat2)
  conv5 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv5)

  outputs = Conv2D(1, 1, activation = 'sigmoid')(conv5)

  model = Model(inputs = inputs, outputs = outputs)
  return model

input_shape = train_images[0].shape
model = unet_model(input_shape)

# model.complie(optimizer = Adam(lr = 1e-4), loss='binary_crossentropy', metrics=['accuracy'])

ValueError: ignored